In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import torch
from tqdm import tqdm
import random

import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Conv1D, MaxPooling1D, Dropout, BatchNormalization

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from lstm_pipe import *
from globals import *
from preprocess import *
from sequential_models import *
from models import *

In [ ]:
(X_train, y_train, X_val, y_val, X_oos, y_oos), scaler = preprocess_data()


Loading datasets. . .
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3289446 entries, (Timestamp('1975-01-31 00:00:00'), 10006) to (Timestamp('2023-12-31 00:00:00'), 93436)
Data columns (total 222 columns):
 #    Column                Non-Null Count    Dtype         
---   ------                --------------    -----         
 0    gvkey                 3132584 non-null  string        
 1    datadate              2880080 non-null  datetime64[ns]
 2    primary               3289446 non-null  bool          
 3    exchcd                3289200 non-null  float64       
 4    me                    3197832 non-null  float64       
 5    ret                   3193500 non-null  float64       
 6    exret                 3193500 non-null  float64       
 7    rf                    3289446 non-null  float64       
 8    age                   3289446 non-null  float64       
 9    mispricing_mgmt       2399702 non-null  float64       
 10   mispricing_perf       2937642 non-null  float64    

100%|██████████| 219/219 [05:13<00:00,  1.81s/it]


Imputing median for each permno. . .



100%|██████████| 26838/26838 [4:06:48<00:00,  1.83it/s]  



Normalisation of vals. . .

Processing training df. . .


14357it [6:46:47,  1.36it/s]  